In [69]:
import requests
import pandas as pd
import numpy as np
import folium
from autobahn.autobahn import get_warnings
from autobahn.car_model import Car
from autobahn.autobahn import map_plot
from autobahn.autobahn import TrafficWarning
from autobahn.autobahn import calculate_traffic_length


In [70]:
url_bab = "https://verkehr.autobahn.de/o/autobahn"

warnings_a7 = get_warnings('A7')

In [71]:
osrm_url = "http://router.project-osrm.org/route/v1/driving/"
start = "9.346188,54.772891"  # Flensburg lng, lat
# end = "9.479746,51.312711"  # Kassel
end = "10.659156,47.568033"# Fuessen
route_url = f"{osrm_url}{start};{end}?overview=full&geometries=geojson"
route_response = requests.get(route_url)
route_data = route_response.json()
route_coords = route_data['routes'][0]['geometry']['coordinates']
route_df = pd.DataFrame(route_coords, columns=['long', 'lat'])

### OSM Data

In [ ]:
print(f"distance: {route_data['routes'][0]['distance'] / 1000:.2f} km")
print(f"duration: {route_data['routes'][0]['duration'] / 3600:.2f} h")

In [ ]:
car = Car(start_location=(54.793743, 9.446996))  # Flensburg
for coord in route_coords:
    car.drive_to((coord[1], coord[0]))

car.distance_traveled

In [ ]:
route_df = route_df.assign(speed=120)
map_plot([route_df], on="speed")

### Assign traffic information

In [ ]:
[print(data["title"]) for data in warnings_a7]

In [ ]:
route_df.columns

In [77]:
from haversine import haversine, haversine_vector

threshold = .1 # km

In [ ]:
for warning in warnings_a7:
    traffic_warning = TrafficWarning(warning)

    gps_coords_warnings = [(row1['lat'], row1['long']) for _, row1 in traffic_warning.geo_df.iterrows()]
    gps_coords_route = [(row2['lat'], row2['long']) for _, row2 in route_df.iterrows()]
    distance_matrix = haversine_vector(gps_coords_warnings, gps_coords_route, comb=True)
    if distance_matrix.min() < threshold:
        print(f"{traffic_warning.title} with delay: {traffic_warning.delayTimeValue}, speed: {traffic_warning.averageSpeed}, distance: {calculate_traffic_length(traffic_warning.geo_df)}")
        ind_route_traffic = route_df["speed"].where(distance_matrix.min(axis=1) < threshold).dropna().index
        route_df.loc[ind_route_traffic, "speed"] = float(traffic_warning.averageSpeed) if traffic_warning.averageSpeed else 80
    # break
    # print(f"Warning: {traffic_warning.title} is part of route: {is_part_of_route}")



In [83]:
import numpy as np

# Generate random fluctuation values
fluctuation = np.random.normal(0, 2, len(route_df))

# Add fluctuation to speed
route_df['speed + fluc'] = route_df['speed'].astype(float) + fluctuation

In [ ]:
route_df["speed + fluc"].plot()

In [ ]:
map_plot([route_df], on="speed")